# 并发与并行
- 使用内置的Queue类型来完成队列
- 使用Lock保证线程安全

In [ ]:
from queue import Queue
# Queue的get方法在拿到新数据前会一直阻塞，直到新数据被放入
my_queue= Queue()
print(f'Consumer Waiting')
def get_func():
    try:
        data = my_queue.get(timeout=3)
    except Exception:
        print(f'get超时, 规定时间内没有拿到数据')
    else:
        print(f'get : {data}')


- 在规定时间内，即使数据在get方法调用后进入，只要没超时就能返回

In [ ]:
from threading import Thread
thread = Thread(target=get_func)
thread.start()

my_queue.put(f'我是数据')

- 限定Queue的最大值可以解决管道的拥堵问题

In [2]:
import time
from threading import Thread
from queue import Queue
my_queue = Queue(1)
def consumer():
    time.sleep(0.1)
    d1 = my_queue.get()
    print(f'Consumer get {d1}')
    d2 = my_queue.get()
    print(f'Consumer get {d2}')
    print(f'Consumer done')
thread = Thread(target=consumer)

my_queue.put(f'data1')
thread.start()
print(f'put data1')
my_queue.put(f'data2', timeout=1)
print(f'put data2')
my_queue.put(f'data3',timeout=1)
print(f'put data3')
thread.join()

put data1
Consumer get data1
put data2
Consumer get data2
Consumer done
put data3


- 对Queue的task_done和join方法完成对queue的状态访问，获取其元素是否全部被加工

子线程task_done被执行后 主线程的.join解除拥塞 否则一直等待

In [1]:
from queue import Queue
import time
from threading import Thread
in_queue = Queue()
def consumer():
    print(f'Consumer waiting')
    work = in_queue.get()
    print(f'Consumer working')
    # Doing
    time.sleep(3)
    print(f'Consumer done')
    in_queue.task_done()
thread = Thread(target=consumer)
thread.start()
print(f'Producer Putting')
in_queue.put(f'Here data be putted')
print(f'Producer waiting')
in_queue.join()
print(f'Producer done')
thread.join()

Consumer waitingProducer Putting
Producer waiting

Consumer working
Consumer done
Producer done


In [2]:
# 自定义一个队列，实现close方法彻底结束任务
from collections.abc import Callable
from typing import Any, Iterable, Mapping


class CloseableQueue(Queue):
    SENTILE = object()
    def close(self):
        self.put(self.SENTILE)
    
    def __iter__(self):
        while True:
            item = self.get()
            try:
                if item is self.SENTILE:
                    return
                yield item
            finally:
                self.task_done()  # 标记get的目标已经处理
# 重新定义工作线程, 当被处理的队列元素用尽就退出线程
class StoppableWorker(Thread):
    def __init__(self, func, in_queue, out_queue, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.func = func
        self.in_queue = in_queue
        self.out_queue = out_queue

    def run(self) -> None:
        for item in self.in_queue:
            result = self.func(item)
            self.out_queue.put(result)



In [ ]:
download_queue = CloseableQueue()
resize_queue = CloseableQueue()
upload_queue = CloseableQueue()
done_queue = CloseableQueue()
# 建立工作线程列表
threads = [StoppableWorker(download, download_queue, resize_queue),
          StoppableWorker(resize, resize_queue, upload_queue),
          StoppableWorker(upload, upload_queue, out_queue=done_queue)]
# 打开工作线程
for thread in threads:
    thread.start()
# 在第一个线程管道放1000个元素
for _ in range(1000):
    download_queue.put(object())
# 停止download的装入
download_queue.close()
# 等待queue元素被全部处理
download_queue.join()
resize_queue.close()
resize_queue.join()
upload_queue.close()
upload_queue.join()
# 等待所有线程停止
for thread in threads:
    thread.join()

- 上个方法中，主线程要逐次等待四个queue的处理操作，我们还可以在每个queue处理时使用多线程

In [ ]:
def start_threads(count, *args):
    threads = [StoppableWorker(*args) for _ in range(count)] 
    for thread in threads:
        thread.start()
    return threads

def stop_threads(closeable_queue: CloseableQueue, threads):
    for _ in threads:
        # [x] 每个线程的queue都需要执行close方法才能退出
        closeable_queue.close()
    closeable_queue.join()
    for thread in threads:
        thread.join()

download_queue = CloseableQueue()
resize_queue = CloseableQueue()
upload_queue = CloseableQueue()
done_queue = CloseableQueue()

download_threads = start_threads(3, download, download_queue, resize_queue)
resize_threads = start_threads(4, resize, resize_queue, upload_queue)
upload_threads = start_threads(4, upload, upload_queue, done_queue)

for _ in range(1000):
    download_queue.put(object())
stop_threads(download_queue, download_threads)
stop_threads(resize_queue, resize_threads)
stop_threads(upload_queue, upload_threads)

# 生命周期游戏的并发处理
Conway's Game of Life 时一个经典的有限状态自动机，规则如下
1. 在任意长宽的二维网格中，每个单元格只有两种状态(Alive; Empty)
2. 要考虑单元格周围有多少处于存活状态的单元格

In [4]:
ALIVE = '*'
EMPTY = '-'

class Grid:
    def __init__(self, height, width) -> None:
        self.height = height
        self.width = width
        self.rows = []
        for _ in range(self.height):
            self.rows.append([EMPTY] * self.width)
    
    def get(self, y, x):
        return self.rows[y % self.height][x % self.width]

    def set(self, y, x, state):
        self.rows[y % self.height][x % self.width] = state
    
    def __str__(self) -> str:
        return '\n'.join(["".join(row) for row in self.rows])
        

grid = Grid(5, 9)
grid.set(0, 3, ALIVE)
grid.set(1, 4, ALIVE)
grid.set(2, 2, ALIVE)
grid.set(2, 3, ALIVE)
grid.set(2, 4, ALIVE)
print(grid)

---*-----
----*----
--***----
---------
---------
